In [5]:
import pandas as pd
from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.time import StagedActivation
from mesa.space import MultiGrid
import os

In [6]:
# Voter Agent
class Voter(Agent):
    def __init__(self, unique_id, model, attributes):
        super().__init__(unique_id, model)
        self.attributes = attributes

    def evaluate_candidate(self):
        min_distance = float('inf')
        chosen_candidate = None

        for candidate in self.model.candidates:
            # Calculate the weighted Euclidean distance
            distance = sum(
                self.attributes[f"{attr}-weight"] * ((self.attributes[attr] - getattr(candidate, attr)) ** 2)
                for attr in [
                    "government-intervention",
                    "government-control-of-economy",
                    "free-trade",
                    "protectionism",
                    "pro-choice",
                    "tax-increase-to-offset-debt-and-deficit",
                    "crime-punishment-level",
                    "military-hawkishness",
                    "non-military-interventions",
                    "affirmative-action",
                    "prayer-in-schools",
                    "market-solution-to-healthcare",
                    "moralistic-law",
                ]
                if attr in self.attributes and f"{attr}-weight" in self.attributes
            )
            # Take the square root of the weighted squared differences
            distance = distance ** 0.5

            if distance < min_distance:
                min_distance = distance
                chosen_candidate = candidate

        return chosen_candidate.unique_id  # Return the chosen candidate's unique ID




# Candidate Agent
class Candidate(Agent):
    def __init__(self, unique_id, model, attributes):
        super().__init__(unique_id, model)
        for attr, value in attributes.items():
            setattr(self, attr, value)
        self.attributes = attributes

In [7]:
# Main Model
class VotingModel(Model):
    def __init__(self, turtles_data, candidate_attributes, info_level_threshold=0):
        super().__init__()  # Call the parent class constructor
        self.schedule = RandomActivation(self)
        self.candidates = []

        # Add Candidate Agents
        for i, attrs in enumerate(candidate_attributes):
            candidate = Candidate(f"Candidate_{i}", self, attrs)
            self.schedule.add(candidate)
            self.candidates.append(candidate)

        # Add Voter Agents
        for _, row in turtles_data.iterrows():
            if row["information-level"] >= info_level_threshold:
                attributes = {
                    "government-intervention": row["government-intervention"],
                    "government-control-of-economy": row["government-control-of-economy"],
                    "free-trade": row["free-trade"],
                    "protectionism": row["protectionism"],
                    "pro-choice": row["pro-choice"],
                    "tax-increase-to-offset-debt-and-deficit": row["tax-increase-to-offset-debt-and-deficit"],
                    "crime-punishment-level": row["crime-punishment-level"],
                    "military-hawkishness": row["military-hawkishness"],
                    "non-military-interventions": row["non-military-interventions"],
                    "affirmative-action": row["affirmative-action"],
                    "prayer-in-schools": row["prayer-in-schools"],
                    "market-solution-to-healthcare": row["market-solution-to-healthcare"],
                    "moralistic-law": row["moralistic-law"],
                    "government-intervention-weight": row["government-intervention-weight"],
                    "government-control-of-economy-weight": row["government-control-of-economy-weight"],
                    "free-trade-weight": row["free-trade-weight"],
                    "protectionism-weight": row["protectionism-weight"],
                    "pro-choice-weight": row["pro-choice-weight"],
                    "tax-increase-to-offset-debt-and-deficit-weight": row["tax-increase-to-offset-debt-and-deficit-weight"],
                    "crime-punishment-level-weight": row["crime-punishment-level-weight"],
                    "military-hawkishness-weight": row["military-hawkishness-weight"],
                    "non-military-interventions-weight": row["non-military-interventions-weight"],
                    "affirmative-action-weight": row["affirmative-action-weight"],
                    "prayer-in-schools-weight": row["prayer-in-schools-weight"],
                    "market-solution-to-healthcare-weight": row["market-solution-to-healthcare-weight"],
                    "moralistic-law-weight": row["moralistic-law-weight"],
                }
                voter = Voter(row["who"], self, attributes)
                self.schedule.add(voter)

        # Data Collector
        self.datacollector = DataCollector(
            agent_reporters={
                "Vote": lambda agent: agent.evaluate_candidate() if isinstance(agent, Voter) else None,
                **{f"column_{i}": lambda agent, i=i: agent.attributes.get(f"column_{i}", None) for i in range(18, 44)}
    }
)


    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
if __name__ == "__main__":
    # Load your turtles_data DataFrame
    base_dir = "C:/Users/hugov/Github/Bachelor-project/Data/CSV"

    # Define the ranges for the file names
    range_1 = range(1, 26)
    range_2 = [500, 750]

    # Dictionary to store the data frames
    turtles_data = {}

    # Loop over the ranges and read the files dynamically
    for i in range_1:
        for j in range_2:
            # Construct the file name
            file_name = f"TURTLES_{i}_{j}.csv"
            file_path = os.path.join(base_dir, file_name)
        
            # Read the CSV file and store it in the dictionary
            if os.path.exists(file_path):
                data_name = f"Turtles{i}_{j}"
                turtles_data[data_name] = pd.read_csv(file_path)
            else:
                print(f"File not found: {file_path}")

    # Combine all DataFrames into one or use a specific DataFrame
    combined_turtles_data = pd.concat(turtles_data.values(), ignore_index=True)  # Option 1
    specific_turtles_data = turtles_data.get("Turtles1_500")                  # Option 2

    # Define attributes for the candidates
    candidate_1_attributes = { # To demonstrate extreme measures, this will be the "good" candidate, in that case. When more realistic it will reflect the policies of Donald Trump as best as possible.
        "government-intervention": 5, # Trump: 25, he wants certain decisions to be left up to the states and doesn't want to impose on free speech, but he still wants some intervention in terms of immigration and against election fraud.
        "government-control-of-economy": 5, # Trump: 50, Trump's economic policy has been described as neo-mercantalist, which centralises monetary funds and controls capital movement. However, he has not passed legilation that directly affected pricing or the like.
        "free-trade": 95, # Trump: 30, Trump deems that free and open trade cannot happen unless america comes out on top, always.
        "protectionism": 5, # Trump: 90, it is the hallmark of Trumps campaign and presidency
        "pro-choice": 100, # Trump: 15, he wants this to be left up to the states and will not sign any federal legislation on this. But he has uttered pro-life sympathies and most republicans echo this sentiment.
        "tax-increase-to-offset-debt-and-deficit": 95, # Trump: 5, one of Trump's main policies are tax-reliefs.
        "crime-punishment-level": 10, # Trump: 70, Trump wants to refund the police and increase penalties for assaults on law-enforcement.
        "military-hawkishness": 10, # Trump: 30, he is adament on pulling the USA out of every war and stopping every war going on. Howver, he is not shy to use the military to enforce peace, when he deems it necessary and has threatened with military might in the past. He also wants the other NATO countries to contribute their fair share.
        "non-military-interventions": 90, #Trump: 70, Trump boasts about his good relations with world leaders and says he prefers to talk it out instead of going to war.
        "affirmative-action": 95, #Trump: 25, he wants to cut funding to program teaching critical race theory and gender ideology to children. He has however secured funding to historically black universities and wants to open civil rights law suits against school districts engaged in race-based discrimination.
        "prayer-in-schools": 5, #Trump: 80, Trump wants to bring back prayer in schools, it is unclear if he wants this to be mandatory or he just supports the right to pray in school.
        "market-solution-to-healthcare": 90, # Trump: 70, He has opened up competition in the healthcare market and lowered drug prices. However, he was not interested in companies like Marc Cuban's Cost Plus Drugs, helping in creating transparency and lowering drug prices.
        "moralistic-law": 5 # Trump: 75, Trump and his administration is sponsoring bills based on christian religious values.
    }

    candidate_2_attributes = { # To demonstrate extreme measures, this will be the "bad" candidate. When more realistic it will reflect the policies of Kamala Harris as best as possible.
        "government-intervention": 95, # Harris: 40, She will introduce measures to build up the middle class and introduce nationwide abortion legislation and lowering cost for everyday americans
        "government-control-of-economy": 95, # Harris: 65, although she favors a market economy, she will and has passed legislation, lowering costs and ensuring tax credits.
        "free-trade": 5, # Harris: 75, she is globally minded and favors a market economy, but still wants to have some control of the trade and crack down on anti-competetive practices
        "protectionism": 95, # Harris: 15, she does want to invest in america, but has a strong global mindset
        "pro-choice": 0, # Harris: 95, she is very pro-choice and wants legislation to protect the freedom to choose and the health of mothers
        "tax-increase-to-offset-debt-and-deficit": 5, # Harris: 80, she wants to lower the taxes for the middle class but raise taxes for anyone making more than 400k USD and have a special billionaire tax and quadruble the tax on stock buybacks.
        "crime-punishment-level": 95, # Harris: 20, Harris famously declined going after the death penalty as district attorney, but still has one of the highest conviction rates in California as AG.
        "military-hawkishness": 95, # Harris: 60, Harris and Biden has funded the two wars raging during their administration, but they still pulled troops out of Afghanistan, although very messily.
        "non-military-interventions": 5, # Harris: 40, Harris condems the use of military domestically, but is in favor of using it abroad and has reaffirmed commitment to NATO.
        "affirmative-action": 5, # Harris: 80, Harris has invested specifically in higher education for minorities, but not sponsored definitive legislation on the matter.
        "prayer-in-schools": 95, # Harris: 20, Harris is not opposed to prayer in schools, if the student feels like praying, but she is opposed to making it mandatory.
        "market-solution-to-healthcare": 5, # Harris: 80, Harris has taken on Big Pharma and helped introduce market solutions and even been open to the transparency that Cost Plus Drugs would offer.
        "moralistic-law": 95 # Harris: 30, the retoric by Harris and the general consensus among democrats is somewhat morally motivated on the abortion front and has elements of "woke" influence, but there is no specific legislation proposed on the matter.
    }
    
    # Initialize the model with the desired DataFrame
    model = VotingModel(specific_turtles_data, [candidate_1_attributes, candidate_2_attributes])  # Use combined or specific

    # Run the model for 10 steps
    for i in range(10):
        model.step()

    # Save collected data to a CSV file
    agent_data = model.datacollector.get_agent_vars_dataframe()

    # Convert agent data to a format suitable for merging
    agent_data.reset_index(inplace=True)
    agent_data.rename(columns={"AgentID": "who"}, inplace=True)  # Ensure matching key for merging

    # Merge original turtles_data with the collected data
    final_data = pd.merge(specific_turtles_data, agent_data, on="who", how="left")

    # Save to CSV
    final_data.to_csv("Data/Voting_results/voting_results.csv", index=False)
    print("Voting results saved to voting_results.csv")



File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_3_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_4_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_5_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_6_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_7_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_8_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_9_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_10_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_11_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_12_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_13_750.csv
File not found: C:/Users/hugov/Github/Bachelor-project/Data/CSV\TURTLES_14_750.csv
File not fo